In [86]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
import torch
from min_llm.models.causal_llm import TransformerLM
from min_llm.decoding import Decoder
from transformers import AutoTokenizer
from min_llm.config import DecoderLMConfig, TransformerConfig, SelfAttentionConfig

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")

In [88]:
decoder = Decoder(tokenizer= tokenizer, temperature = 1.0, top_p = 0.2, device = device)

In [ ]:
model_config = DecoderLMConfig(d_model=512,
                                   vocab_size = 50257,
                                   context_length = 256,
                                   n_layers = 4,
                                   pos_embedding_type="rope",
                                   dropout = 0.1,
                                   transformer=TransformerConfig(norm_position = "pre",
                                                                 norm_type = "rms",
                                                                 attn = SelfAttentionConfig(
                                                                     n_heads=16, 
                                                                     dropout_attn=0.1))) 

In [ ]:
model = TransformerLM(model_config,device=device)
checkpoint = torch.load(f"../experiments/tinystories/szlm82da/checkpoints/best_model.pth", map_location=device)
model.load_state_dict(checkpoint["model_state_dict"])
model.eval()

In [ ]:
input_string = r"There once was a little boy who was very naughty"
output = decoder.decode(model, input_string, max_tokens = 200)
print(output)